<h1> <font color=red> Engenharia 2019.2 - Ciência dos dados </font> </h1>

<h2> Projeto 2 - Classificador Automático de Sentimento</h2>

<h3> Alunos: </h3>
<ul>
    <li>Cicero Tiago Carneiro Valentim</li>
    <li>Luiz Felipe Lazzaron</li>
    <li>Marcos Vinícius</li>
</ul>

<h3> Sumário: </h3>
<ol>
   <li> Introdução </li>
   <p></p>
   <li>Preparando Ambiente no Jupyter</li>
   <p></p>
   <li>Autenticando no Twitter</li>
   <p></p>
   <li>Escolha de um produto</li>
   <p></p>
   <li>Classificando as mensagens na coragem</li>
   <p></p>
   <li>Montando o Classificador Naive-Bayes</li>
   <p></p>
   <li>Limpeza e Tratamento de Dados</li>
   <ul>
       <li>7.1 Função Remove;</li>
       <li>7.2 Função SpaceCleaning;</li>
       <li>7.3 Função TratamentodoGrupo;</li>
       <li>7.4 Função Stemming;</li>
       <li>7.5 Função RemoveStopWords;</li>
       <li>7.6 Tratando os Dados da Base de Treinamento;</li>
   </ul>
   <p></p>
   <li>Naive Bayes e Laplace Smoothing</li>
   <ul>
       <li>8.1 Função lista_sem_repetição</li>
       <li>8.2 Função laplace</li>
       <li>8.3 Função numero_de_vezes</li>
       <li>8.4 Função conta_palavras</li>
   </ul>
   <p></p>
   <li>Verificando a performance na Planilha Testes</li>
   <ul>
       <li>9.1 Função limpeza_tweet;</li>
       <li>9.2 Função analisa_frase;</li>
       <li>9.3 Função identifica_valor_maximo;</li>
       <li>9.4 Função resultado;</li>       
   </ul>
   <p></p>
   <li>Conclusão</li>
   <p></p>
   <li>Referências</li>
</ol>

## 1. Introdução

<p>
    No presente trabalho, visa-se selecionar tweets relevantes para a classificação de postagens que possuem sinais de depressão por parte de seus autores. Nesse contexto, os integrantes dividiram os tweets em três categorias:
</p>
<ul>
    <li> 1. Irrelavantes ou Neutros. Tweets que não indicam quaisquer indício de depressão;</li>
    <li> 2. Relevantes. Tweets que possuem relevância na análise de indicação de depressão;</li>
    <li> 3. Possuem interesse no assunto. Tweets que não demonstram doença mental, mas indicam um interresse no tema e na prevenção de suicídio;</li>
</ul>
<p>
    Desse modo, com tal classificação, pode-se desenvolver um marketing específico para os públicos 2 e 3; para os relevantes, pode-se oferecer auxílio e suporte psicológico; para aqueles que possuem interesse no tema "depressão", pode-se direcionar materiais que ensinem a como lidar com pessoas com depressão.
</p>

## 2. Preparando o ambiente no jupyter:

In [2]:
!pip install emoji
!pip install nltk
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from IPython.display import clear_output
from time import sleep
import script
from emoji import UNICODE_EMOJI
import emoji
import re 
import functools
import operator
import nltk 
from nltk.stem import RSLPStemmer
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('punkt');

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 3. Autenticando no  Twitter

In [3]:
#Conta: ***@MarcosV96118169***

#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @MarcosV96118169

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

## 4. Escolha de um produto

<ul>
Produto escolhido: <b>Depressão</b>

Quantidade mínima de mensagens capturadas: <b>750</b>

Quantidade mínima de mensagens para a base de treinamento: <b>300</b>

Filtro de língua: <b>pt</b>
</ul>

## 5. Classificando as mensagens na coragem

Esta etapa foi manual e foi feita diretamente no Excel.

#### Lendo arquivos xlsx:

In [4]:
planilha_treinamento = pd.read_excel('depressao.xlsx', "Treinamento")
planilha_teste = pd.read_excel('depressao.xlsx', "Teste")
pd.set_option('display.max_colwidth', -1)

## 6. Montando o Classificador Naive-Bayes

<p>Primeiramente, iremos trabalhar com o <i>dataframe</i> treinamento, criando três banco de dados:</p>
<ul>
    <li>Grupo 1 - Irrelevantes/Neutros: group_1</li>
    <li>Grupo 2 - Relevantes: group_2</li>
    <li>Grupo 3 - Interesse no Assunto: group_3</li>
</ul>

In [5]:
group_1 = planilha_treinamento[planilha_treinamento["Relevância"] == 1]
group_2 = planilha_treinamento[planilha_treinamento["Relevância"] == 2]
group_3 = planilha_treinamento[planilha_treinamento["Relevância"] == 3]

## 7. Limpeza e Tratamento dos Dados

### 7.1 Função Remove:
<p>Essa função serve para retirarmos do nosso <i>DataFrame</i> qualquer string com as iniciais <b>http</b> e <b>@.</b></p>

In [6]:
def Remove(dataframe):
    data_aux = dataframe
    lista_tweets = list(dataframe.loc[:,"Tweet Text"].values)
    lista_reserva = []
    for i in range(len(lista_tweets)):
        lista_tweets[i] = lista_tweets[i].split()
        for t in range(len(lista_tweets[i])):
            if "http" in lista_tweets[i][t]:
                lista_tweets[i][t] = "http"
        while lista_tweets[i].count("http") != 0:
            lista_tweets[i].remove("http")
        lista_tweets[i] = ' '.join(lista_tweets[i])
    data_aux.loc[:,"Tweet Text"] = lista_tweets
    return data_aux

### 7.2 Função SpaceCleaning:
<p>Essa função serve para quebrarmos nosso texto por palavras e emojis, criando desse modo um array com todas as palavras e mojis contidas dentro do texto. Além disso, retiramos do nosso array algumas pontuações desnecessárias para a análise.</p>

In [7]:
def SpaceCleaning(text):
    punctuation = '[!\-.:?;,''"@/]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    split_emoji = emoji.get_emoji_regexp().split(text_subbed)
    split_whitespace = [substr.split() for substr in split_emoji]
    split = functools.reduce(operator.concat,split_whitespace)
    return split

### 7.3 Função TratamentodoGrupo:
<p>Essa função serve para tratar a base de dados do grupo 1, 2 e 3.</p>

In [8]:
def TratamentodoGrupo(grupo):
    total = []
    for tweet in grupo["Tweet Text"]:
        tweet_splited = SpaceCleaning(tweet)
        total.append(tweet_splited)
    return total

### 7.4 Função Stemming:
<p>Essa função serve para diminuirmos a palavra até a sua raiz, pois assim, conseguimos tratar as palavras originais e suas respectivas derivações de uma mesma maneira.</p>

In [9]:
def Stemming(sentence):
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return phrase

### 7.5 Função RemoveStopWords:
<p>Essa função serve para retirarmos dentro do nosso array algumas palavras que não são interessantes para contabilizarmos uma pontuação na hora de classificar o nosso texto, então mantemos somente as palavras principais.</p>

In [10]:
def RemoveStopWords(sentence):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    phrase = []
    for word in sentence:
        if word not in stopwords:
            phrase.append(word)
    return phrase

In [34]:
stopwords = nltk.corpus.stopwords.words('portuguese')

### 7.6 Tratando os Dados da Base de Treinamento:

In [38]:
#Tratando com a função: Removehttp
a0 = Remove(group_1)
b0 = Remove(group_2)
c0 = Remove(group_3)

#Tratando com a função: TratamentoGrupo
a1 = TratamentodoGrupo(a0)
b1 = TratamentodoGrupo(a0)
c1 = TratamentodoGrupo(a0)

#Tratando com a função: Stemming
i = 0
a2 = len(a1)*[0]
while i < len(a1):
    a2[i] = Stemming(a1[i])
    i+=1
    
h = 0
b2 = len(b1)*[0]
while h < len(b1):
    b2[h] = Stemming(b1[h])
    h+=1

j = 0
c2 = len(c1)*[0]
while j < len(c1):
    c2[j] = Stemming(c1[j])
    j+=1

#Tratando com a função: RemoveStopWords
q = 0
a3 = len(a2)*[0]
while q < len(a2):
    a3[q] = RemoveStopWords(a2[q])
    q+=1
    
w = 0
b3 = len(b2)*[0]
while w < len(b2):
    b3[w] = RemoveStopWords(b2[w])
    w+=1

l = 0
c3 = len(c2)*[0]
while l < len(c2):
    c3[l] = RemoveStopWords(c2[l])
    l+=1

groups_tweet = [a3,b3,c3];

## 8. Naive Bayes e Laplace Smoothing

### 8.1 Função lista_sem_repetição:
<p> A função abaixo cria uma lista com todas as palavras existentes num grupo sem repetição, adicionando elas na lista total_words. No fim, temos o número de palavras sem repetição, a qual é importante para cálculos futuros envolvendo a expressão de Laplace.
</p>

In [13]:
def lista_sem_repeticao(LISTA,lista):
    resposta = LISTA
    for tweet in lista:
        for palavra in tweet:
            if palavra not in resposta:
                resposta.append(palavra)
    return resposta

In [14]:
total_words = []
lista_sem_repeticao(total_words,groups_tweet[0])
lista_sem_repeticao(total_words,groups_tweet[1])
lista_sem_repeticao(total_words,groups_tweet[2])
N =len(total_words)

### 8.2 Função laplace:
<p> Abaixo, tem-se a função proposta por Laplace, a qual realiza um smoothing nos dados. Ela recebe três parâmetros: 
</p>
<ul>
    <li>Número de vezes que a palavra aparece no grupo;</li>
    <li>Número de elementos contido no grupo (1, 2 ou 3);</li>
    <li>Número de elementos sem repetição em todas os grupos</li>    
</ul>

In [15]:
def laplace(num_vezes_no_grupo, num_elementos_do_grupo, num_elementos_sem_repeticao):
    probabilidade = (num_vezes_no_grupo + 1 ) / ( num_elementos_do_grupo + num_elementos_sem_repeticao)
    return probabilidade

### 8.3 Função numero_de_vezes:
<p> A função "número de vezes" fornece a quantidade de vezes que a palavra aparece no grupo. Essa função é utilizada juntamente com Laplace. 
</p>

In [16]:
def numero_de_vezes(palavra_analisada,grupo):
    n = 0
    for tweet in grupo:
        for palavra in tweet:
            if palavra ==  palavra_analisada:
                n +=1
    return n

In [17]:
resultados = []
for group in groups_tweet:
    dictionary= {}
    for tweet in group:
        for word in tweet:
            probabilidade = laplace(numero_de_vezes(word,group),len(group),N)
            dictionary[word] = probabilidade
    resultado = pd.Series(dictionary)
    resultados.append(resultado)

### 8.4 Função conta_palavras:
<p> A função palavras vê quantas existem num grupo, o qual é composto por um série de tweets.
</p>

In [18]:
def conta_palavras(grupo):
    n = 0
    for tweet in grupo:
        n += len(tweet)
    return n

In [19]:
N_1 = conta_palavras(groups_tweet[0])
N_2 = conta_palavras(groups_tweet[1])
N_3 = conta_palavras(groups_tweet[2])

## 9. Verificando a performance na Planilha Testes
<p>
Testando classificador com a base de Testes.
</p>

In [20]:
teste = planilha_teste["Tweet Text"]

### 9.1 Função limpeza_tweet:
<p> A função limpa cada twetet.
</p>

In [21]:
def limpeza_tweet(tweet):
    tweet_limpo = SpaceCleaning(str(tweet))
    return tweet_limpo

In [22]:
teste_tweet = []
for tweet in teste:
    teste_tweet.append(limpeza_tweet(tweet))

In [23]:
serie_teste = pd.Series(teste_tweet)

### 9.2 Função analisa_frase:
<p> Como os resultados estão muito pequenos, usou-se a função do logarítmo neperiano, ln(x), para os valores ficarem com números mais adequados para o Python.
</p>

In [24]:
def analisa_frase(frase):
    condicional_grupo1 = 1
    condicional_grupo2 = 1
    condicional_grupo3 = 1
    lista = [condicional_grupo1, condicional_grupo2,condicional_grupo3]
    
    for palavra in frase:
        if (palavra not in resultados[0].index):
            condicional_grupo1 *= laplace(0,N_1,N)
        else: 
            condicional_grupo1 *= resultados[0][palavra]
            
    for palavra in frase:
        if (palavra not in resultados[1].index):
            condicional_grupo2 *= laplace(0,N_1,N) 
        else:
            condicional_grupo2 *= resultados[1][palavra]
            
    for palavra in frase:
        if (palavra not in resultados[2].index):
            condicional_grupo3 *= laplace(0,N_1,N) 
        else:
            condicional_grupo3 *= resultados[2][palavra]
            
    return [condicional_grupo1,condicional_grupo2,condicional_grupo3]

### 9.3 Função identifica_valor_maximo:

In [25]:
def identifica_valor_maximo(lista_de_resultados):
    if lista_de_resultados[0] == max(lista_de_resultados):
        return 1
    elif lista_de_resultados[1] == max(lista_de_resultados):
        return 2
    else:
        return 3

### 9.4 Função resultado:

In [26]:
def resultado1(n):
    return identifica_valor_maximo(analisa_frase(limpeza_tweet(teste[n])))

def resultado2(database_coluna, n):
    return identifica_valor_maximo(analisa_frase(limpeza_tweet(database_coluna[n])))


In [27]:
estimativa = []
for k in range(0,len(teste)):
    estimativa.append(resultado1(k))

planilha_teste["Relevância_Estimada"] = estimativa

Acertos = 0
for n in planilha_teste.index:
    if planilha_teste["Relevância_Prevista"][n] == planilha_teste["Relevância_Estimada"][n]:
        Acertos += 1

In [28]:
planilha_teste["Relevância_Prevista"].value_counts(True)

1    0.540000
3    0.236667
2    0.223333
Name: Relevância_Prevista, dtype: float64

In [29]:
planilha_treinamento["Relevância"].value_counts(True)

1.0    0.492205
2.0    0.285078
3.0    0.222717
Name: Relevância, dtype: float64

In [30]:
planilha_teste["Relevância_Estimada"].value_counts(True)

1    1.0
Name: Relevância_Estimada, dtype: float64

In [31]:
Acertos/len(planilha_teste)

0.54

In [32]:
def return_tweet(quantidade, autorizacao):
    """return_tweet(autorizacao, quantidade)"""
    api = tweepy.API(autorizacao)
    i = 0
    __lista__ = []
    anterior = {'User Name': 'foo',
                'Tweet Created At': 'foo','Tweet Text': 'foo','Relevância': 'foo',
                'User Location': 'foo','Phone Type': 'foo',
                'Favorite Count': 'foo','Retweets':'foo'}
    while i < quantidade:
        for msg in tweepy.Cursor(api.search, q="{0} -filter:retweets".format("depressao"), lang='pt', tweet_mode="extended").items():
            new_msg = {'Tweet Text': msg.full_text.lower()}
            i += 1
            if  new_msg["Tweet Text"] not in __lista__:
                __lista__.append(new_msg)
                if resultado2(list(new_msg["Tweet Text"]),0) == 1:
                    __resultado__ = "Irrelevante"
                elif resultado2(list(new_msg["Tweet Text"]),0) == 2:
                    __resultado__ = "Preocupante"
                elif resultado2(list(new_msg["Tweet Text"]),0) == 3:
                    __resultado__ = "Demonstra interesse pelo tema"
                print("[{0}]\nClassificação:\t{1}\nTexto:\t{2}".format(i,__resultado__, new_msg["Tweet Text"]))
            print("\t -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-\n\n")
            sleep(4.5)
            if i >= quantidade:
                break
        print("[Fim do Programa]")

In [39]:
return_tweet(50,auth) # return_tweet(quantidade, autenticação)

[1]
Classificação:	Irrelevante
Texto:	eu era tão segura de mim mesma que parece que sou outra pessoa depois q me afundei nessa merda dessa depressão
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[2]
Classificação:	Irrelevante
Texto:	apenas um nerdcast com a agatha e a portuguesa curaria minha depressão
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[3]
Classificação:	Irrelevante
Texto:	depressão tá me dando uma surra hoje, não sei o que fiz pra merecer se tomei meus remédios direitinho :c
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[4]
Classificação:	Irrelevante
Texto:	uma vez achei q tava com depressao, mas era só tpm msm https://t.co/yiqqksjuj1
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[5]
Classificação:	Irrelevante
Texto:	engraçado que as pessoas me olham sorrindo e não imaginam, que eu posso estar em depressão
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[6]
Classificação:	Irrelevante
Texto:	na moral depressão só aparece pra gente perder tempo
	 -=-=-=-=-=

KeyboardInterrupt: 

## 10. Conclusão

<p>Diante do que foi observado, pode-se verificar que a base de dados forneceu um acerto em torno de 54%. Ou seja, mesmo que o resultado não seja tão expressivo, o verificador pode ser útil na identificação de pessoas que possuem indicativo de depressão. Nesse contexto, pode-se pensar em melhorias na base de dados, afim de otimizar o acerto. Um fator que foi utilizado para melhorar o tratamentos dos tweets foi o stemming, que transforma palavras de mesmo radical, bem como a exclusão de preposições e conectivos, que não auxiliam no julgamento do tweet. Análogo a essa melhoria, pode-se pensar em outras maneiras de tratar as palavras, como substituir palavras que são sinônimos.</p>

## 11. Referências

<li>[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)</li>

<li>[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/)</li>